# Retriever

## Imports

In [2]:
import os
import re
from typing import List

import chromadb
from dotenv import load_dotenv, find_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
DATABASE_PATH = "../../Database/"
EMBEDDING_MODEL = "text-embedding-ada-002"
CHUNK_SIZE = 1800
CHUNKING_LIBRARY = "Unstructured"                                       # "RCTS" only contains 17 files, "Unstructured" contains 108 files
METHOD = "by_title" if CHUNKING_LIBRARY == "Unstructured" else ""       # "by_title" or "basic" - only relevant for Unstructured


path_to_db = os.path.join(DATABASE_PATH, f"{CHUNKING_LIBRARY}", f"{METHOD}", f"{EMBEDDING_MODEL}")


client = chromadb.PersistentClient(
    path=path_to_db,
)


vectorstore = Chroma(
    collection_name=f"collection_{CHUNK_SIZE}",
    embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=EMBEDDING_MODEL),
    client=client,
    create_collection_if_not_exists=False,
)

In [4]:
def pretty_output(text: str, words_per_line: int = 10) -> str:
    """ Prettier output for the text with a given number of words per line.

    Args:
        text (str): Text to be formatted.
        words_per_line (int, optional): Number of words per line. Defaults to 10.

    Returns:
        str: Formatted text.
    """
    text = re.sub(r"\s+", " ", text)
    split_text = text.split(" ")

    text = ""
    for i, word in enumerate(split_text, 1):
        text += word + " "
        if i % words_per_line == 0:
            text += "\n"

    return text

## Create Retriever

In [9]:
QUERY = "Nach welchen Paragraph kann ich Leistungen ablehnen, wenn Unterlagen fehlen?"

### Default Retriever

An simple retriever that returns the top k documents based on the cosine similarity between the query and the documents.

Watch [here](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/)

In [ ]:
# using the .as_retriever() method to get the retriever object

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    },
)

chunks = retriever.invoke(QUERY)
chunks

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print("-" * 250)
    print(pretty_output(chunk.page_content, 20))
    print("\n")

In [60]:
# # using the .similarity_search() method to get the chunks

# chunks = vectorstore.similarity_search(
#     query=QUESTION,
#     k=5,
# )

# chunks

In [61]:
# for i, chunk in enumerate(chunks):
#     print(f"Chunk {i+1}:")
#     print("-" * 250)
#     print(pretty_output(chunk.page_content, 20))
#     print("\n")

### Multi Query Retriever

Die abstandsbasierte Abfrage von Vektordatenbanken bettet Abfragen in den hochdimensionalen Raum ein (stellt sie dar) und findet ähnliche eingebettete Dokumente auf der Grundlage einer Abstandsmetrik. Die Abfrage kann jedoch zu unterschiedlichen Ergebnissen führen, wenn sich der Wortlaut der Abfrage geringfügig ändert oder wenn die Einbettung die Semantik der Daten nicht gut erfasst. Um diese Probleme manuell zu beheben, wird manchmal ein Prompt-Engineering/-Tuning durchgeführt, was jedoch sehr mühsam sein kann.

Der MultiQueryRetriever automatisiert den Prozess der Promptabstimmung, indem er einen LLM verwendet, um mehrere Abfragen aus verschiedenen Perspektiven für eine gegebene Benutzereingabe zu generieren. Für jede Abfrage wird ein Satz relevanter Dokumente abgerufen und die eindeutige Vereinigung aller Abfragen genommen, um einen größeren Satz potenziell relevanter Dokumente zu erhalten. Durch die Generierung mehrerer Perspektiven auf dieselbe Frage kann der MultiQueryRetriever einige der Beschränkungen der abstandsbasierten Suche abmildern und eine reichhaltigere Menge an Ergebnissen erhalten.

Watch [here](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/)

`.from_llm` is using following prompt to generate 3 different queries for the same question.

```python
DEFAULT_QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is 
    to generate 3 different versions of the given user 
    question to retrieve relevant documents from a vector  database. 
    By generating multiple perspectives on the user question, 
    your goal is to help the user overcome some of the limitations 
    of distance-based similarity search. Provide these alternative 
    questions separated by newlines. Original question: {question}""",
)
```

#### Retrieving Without Chain

In [62]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0,  # to avoid randomness
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [63]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [64]:
unique_docs = retriever_from_llm.invoke(QUERY)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Kann ein Arbeitgeber einen Zuschuss zur Ausbildungsvergütung beantragen, und wenn ja, welche Bedingungen müssen dafür erfüllt sein?  ', 'Welche Voraussetzungen müssen erfüllt sein, damit ein Arbeitgeber einen Zuschuss zur Ausbildungsvergütung erhalten kann?  ', 'Gibt es die Möglichkeit, dass ein Arbeitgeber einen Zuschuss zur Ausbildungsvergütung erhält, und welche Kriterien sind dafür entscheidend?']


5

In [ ]:
unique_docs

#### Retrieving as Chain

all this stuff is happening inside of `.from_llm` method.

In [66]:
# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0,  # to avoid randomness
)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# Other inputs
question = "What are the approaches to Task Decomposition?"

In [67]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), 
    llm_chain=llm_chain, 
    parser_key="lines"  # "lines" is the key (attribute name) of the parsed output
)  

# Results
unique_docs = retriever.invoke(QUERY)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Welche Voraussetzungen müssen erfüllt sein, damit ein Arbeitgeber einen Zuschuss zur Ausbildungsvergütung erhalten kann?', '2. Ist es möglich, dass ein Arbeitgeber einen Zuschuss zur Ausbildungsvergütung beantragt, und wenn ja, welche Bedingungen müssen dafür gelten?', '3. Unter welchen Bedingungen kann ein Zuschuss zur Ausbildungsvergütung für Arbeitgeber gewährt werden?', '4. Gibt es spezielle Richtlinien oder Voraussetzungen, die ein Arbeitgeber beachten muss, um einen Zuschuss zur Ausbildungsvergütung zu beantragen?', '5. Welche Kriterien müssen erfüllt sein, damit ein Zuschuss zur Ausbildungsvergütung für einen Arbeitgeber genehmigt wird?']


9

In [ ]:
unique_docs

### Contextual Compression Retriever

Eine Herausforderung bei der Abfrage besteht darin, dass Sie in der Regel nicht wissen, mit welchen spezifischen Abfragen Ihr Dokumentenspeichersystem konfrontiert wird, wenn Sie Daten in das System einspeisen. Das bedeutet, dass die für eine Abfrage wichtigsten Informationen möglicherweise in einem Dokument mit einer Menge irrelevantem Text vergraben sind. Die Weiterleitung des gesamten Dokuments durch Ihre Anwendung kann zu teureren LLM-Aufrufen und schlechteren Antworten führen.

Die kontextuelle Komprimierung soll hier Abhilfe schaffen. Die Idee ist einfach: Anstatt die abgerufenen Dokumente sofort unverändert zurückzugeben, können Sie sie unter Verwendung des Kontexts der gegebenen Anfrage komprimieren, so dass nur die relevanten Informationen zurückgegeben werden. „Komprimieren“ bezieht sich hier sowohl auf die Komprimierung des Inhalts eines einzelnen Dokuments als auch auf das Herausfiltern von Dokumenten im Ganzen.

Um den Contextual Compression Retriever zu verwenden, benötigen Sie:
- einen Basis-Retriever
- einen Dokumentenkompressor

Der Contextual Compression Retriever übergibt Abfragen an den Basis-Retriever, nimmt die Ausgangsdokumente und leitet sie durch den Document Compressor. Der Document Compressor nimmt eine Liste von Dokumenten und kürzt sie, indem er den Inhalt von Dokumenten reduziert oder Dokumente ganz weglässt.

Watch [here](https://python.langchain.com/v0.2/docs/how_to/contextual_compression/)

### Parent Retriever

Watch [here](https://python.langchain.com/v0.2/docs/how_to/parent_document_retriever/)